In [2]:
!pip install pyvis

   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ------ --------------------------------- 122.9/756.0 kB 2.4 MB/s eta 0:00:01
   ---------------------- ----------------- 419.8/756.0 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------  737.3/756.0 kB 3.9 MB/s eta 0:00:01
   ---------------------------------------  747.5/756.0 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 756.0/756.0 kB 2.8 MB/s eta 0:00:00


DEPRECATION: Loading egg at c:\users\mobasser\anaconda3\lib\site-packages\lightrag_hku-0.0.8-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightrag-hku 0.0.8 requires accelerate, which is not installed.
lightrag-hku 0.0.8 requires aioboto3, which is not installed.
lightrag-hku 0.0.8 requires aiohttp, which is not installed.
lightrag-hku 0.0.8 requires graspologic, which is not installed.
lightrag-hku 0.0.8 requires hnswlib, which is not installed.
lightrag-hku 0.0.8 requires nano-vectordb, which is not installed.
lightrag-hku 0.0.8 requires ollama, which is not installed.
lightrag-hku 0.0.8 requires openai, which is not installed.
lightrag-hku 0.0.8 requires t

In [3]:
# --- Semantic Mind-Map → Interactive HTML (PyVis) ---
import json
from pathlib import Path
import pandas as pd
import networkx as nx
from pyvis.network import Network

# 1) Load your mindmap JSON
data = json.loads(Path("mindmap.json").read_text(encoding="utf-8"))

# 2) Build the graph
G = nx.DiGraph()

def add_node_safe(nid, **attrs):
    if not nid: return
    nid = str(nid)
    if not G.has_node(nid):
        G.add_node(nid, **attrs)
    else:
        G.nodes[nid].update({k:v for k,v in attrs.items() if v not in (None, "")})

# Participants
for p in data.get("participants", []):
    add_node_safe(p["name"], type="participant", label=p["name"])

# Topics / Subtopics + implicit links
for t in data.get("main_topics", []):
    topic = t["topic"]
    add_node_safe(topic, type="topic", label=topic,
                  introduced_by=t.get("introduced_by"),
                  sentiment=t.get("sentiment"))
    if t.get("introduced_by"):
        G.add_edge(t["introduced_by"], topic, relation="introduced", when=t.get("introduced_at"))

    for s in t.get("subtopics", []):
        sub = s["subtopic"]
        add_node_safe(sub, type="subtopic", label=sub, sentiment=s.get("sentiment"))
        G.add_edge(topic, sub, relation="subtopic_of", when=s.get("introduced_at"), stance=s.get("stance"))
        for who in s.get("discussed_by", []):
            if who:
                G.add_edge(who, sub, relation="discussed", when=s.get("introduced_at"), stance=s.get("stance"))

# Explicit relationships
for r in data.get("relationships", []):
    frm, to = r.get("from"), r.get("to")
    if frm and to:
        add_node_safe(frm, type=G.nodes.get(frm, {}).get("type","concept"), label=frm)
        add_node_safe(to,  type=G.nodes.get(to,  {}).get("type","concept"),  label=to)
        G.add_edge(frm, to, relation=r.get("type","related"),
                   when=r.get("initiated_at"), by=r.get("initiated_by"))


# 4) Interactive HTML with PyVis
net = Network(height="800px", width="100%", directed=True, notebook=True, bgcolor="#ffffff", font_color="#222")

# Style maps
type_style = {
    "participant": dict(color="#1E88E5", shape="diamond"),
    "topic":       dict(color="#F9A825", shape="ellipse"),
    "subtopic":    dict(color="#43A047", shape="dot"),
    "concept":     dict(color="#8E24AA", shape="dot"),
}

def tint_by_sentiment(base_hex, sentiment):
    if not sentiment or sentiment == "neutral": return base_hex
    c = base_hex.lstrip("#"); r,g,b = int(c[:2],16), int(c[2:4],16), int(c[4:],16)
    if sentiment == "negative": r,g,b = int(r*0.75), int(g*0.75), int(b*0.75)
    elif sentiment == "positive": r,g,b = min(255,int(r*1.15)), min(255,int(g*1.15)), min(255,int(b*1.15))
    return f"#{r:02x}{g:02x}{b:02x}"

for n, attrs in G.nodes(data=True):
    base = type_style.get(attrs.get("type","concept"), type_style["concept"])
    color = tint_by_sentiment(base["color"], attrs.get("sentiment"))
    info = []
    for k in ("type","introduced_by","sentiment"):
        if attrs.get(k): info.append(f"<b>{k}</b>: {attrs[k]}")
    net.add_node(n, label=attrs.get("label", n), shape=base["shape"], color=color, title="<br>".join(info) or n)

edge_color = {
    "introduced":  "#607D8B",
    "subtopic_of": "#9E9E9E",
    "discussed":   "#90A4AE",
    "support":     "#43A047",
    "extension":   "#1976D2",
    "elaboration": "#6A1B9A",
    "challenge":   "#E53935",
    "related":     "#757575",
}
for u, v, attrs in G.edges(data=True):
    rel = attrs.get("relation") or "related"
    title = [f"<b>relation</b>: {rel}"]
    if attrs.get("stance"): title.append(f"<b>stance</b>: {attrs['stance']}")
    if attrs.get("when"):   title.append(f"<b>at</b>: {attrs['when']}")
    if attrs.get("by"):     title.append(f"<b>by</b>: {attrs['by']}")
    net.add_edge(u, v, color=edge_color.get(rel, "#757575"), title="<br>".join(title), arrows="to")

# Nice defaults
net.show_buttons(filter_=["physics", "interaction"])
net.repulsion(node_distance=220, spring_length=180)

# 5) Save & open
net.show("rough_mindmap_interactive.html")
print("Open mindmap_interactive.html in your browser.")


rough_mindmap_interactive.html
Open mindmap_interactive.html in your browser.


In [6]:
# --- Semantic Mind-Map → Interactive HTML (PyVis, hierarchical & clean) ---

import json
from pathlib import Path
import textwrap
import networkx as nx
from pyvis.network import Network

# 1) Load your mindmap JSON
DATA_PATH = "mindmap.json"  # change if needed
data = json.loads(Path(DATA_PATH).read_text(encoding="utf-8"))

# 2) Build the graph
G = nx.DiGraph()

def add_node_safe(nid, **attrs):
    if not nid:
        return
    nid = str(nid)
    if not G.has_node(nid):
        G.add_node(nid, **attrs)
    else:
        G.nodes[nid].update({k: v for k, v in attrs.items() if v not in (None, "")})

# Participants (level 0)
for p in data.get("participants", []):
    add_node_safe(p["name"], type="participant", label=p["name"], level=0)

# Topics / Subtopics + implicit links
for t in data.get("main_topics", []):
    topic = t["topic"]
    add_node_safe(
        topic,
        type="topic",
        label=topic,
        introduced_by=t.get("introduced_by"),
        sentiment=t.get("sentiment"),
        level=1,
    )
    if t.get("introduced_by"):
        G.add_edge(t["introduced_by"], topic, relation="introduced", when=t.get("introduced_at"))

    for s in t.get("subtopics", []):
        sub = s["subtopic"]
        add_node_safe(sub, type="subtopic", label=sub, sentiment=s.get("sentiment"), level=2)
        G.add_edge(topic, sub, relation="subtopic_of", when=s.get("introduced_at"), stance=s.get("stance"))
        for who in s.get("discussed_by", []):
            if who:
                G.add_edge(who, sub, relation="discussed", when=s.get("introduced_at"), stance=s.get("stance"))

# Explicit relationships (optional cross-links)
for r in data.get("relationships", []):
    frm, to = r.get("from"), r.get("to")
    if frm and to:
        add_node_safe(frm, type=G.nodes.get(frm, {}).get("type", "concept"), label=frm)
        add_node_safe(to,  type=G.nodes.get(to,  {}).get("type", "concept"),  label=to)
        G.add_edge(frm, to, relation=r.get("type", "related"),
                   when=r.get("initiated_at"), by=r.get("initiated_by"))

# 3) Interactive HTML with PyVis
net = Network(height="820px", width="100%", directed=True, notebook=True, bgcolor="#ffffff", font_color="#222")

# Use proper JSON for options; include hierarchical layout + configure panel
net.set_options(json.dumps({
    "layout": {
        "hierarchical": {
            "enabled": True,
            "direction": "UD",
            "sortMethod": "hubsize",
            "levelSeparation": 220,
            "nodeSpacing": 220,
            "treeSpacing": 260
        }
    },
    "physics": {"enabled": False},
    "nodes": {
        "font": {"multi": "md", "size": 14},
        "widthConstraint": {"maximum": 240}
    },
    "edges": {
        "smooth": {"type": "dynamic"},
        "arrows": {"to": {"enabled": True, "scaleFactor": 0.7}}
    },
    "interaction": {
        "hover": True,
        "tooltipDelay": 100
    },
    # VisJS "configure" panel (replacement for pyvis.show_buttons())
    "configure": {
        "enabled": True,
        "filter": ["layout", "interaction", "physics", "edges", "nodes"]
    }
}))

# Visual style
type_style = {
    "participant": dict(color="#1E88E5", shape="diamond"),
    "topic":       dict(color="#F9A825", shape="box"),
    "subtopic":    dict(color="#43A047", shape="ellipse"),
    "concept":     dict(color="#8E24AA", shape="dot"),
}

edge_color = {
    "introduced":  "#607D8B",
    "subtopic_of": "#9E9E9E",
    "discussed":   "#90A4AE",
    "support":     "#43A047",
    "extension":   "#1976D2",
    "elaboration": "#6A1B9A",
    "challenge":   "#E53935",
    "related":     "#757575",
}

def tint_by_sentiment(base_hex, sentiment):
    if not sentiment or sentiment == "neutral":
        return base_hex
    c = base_hex.lstrip("#")
    r, g, b = int(c[:2], 16), int(c[2:4], 16), int(c[4:], 16)
    if sentiment == "negative":
        r, g, b = int(r * 0.75), int(g * 0.75), int(b * 0.75)
    elif sentiment == "positive":
        r, g, b = min(255, int(r * 1.15)), min(255, int(g * 1.15)), min(255, int(b * 1.15))
    return f"#{r:02x}{g:02x}{b:02x}"

def wrap_label(text, width=28, max_lines=3):
    if not isinstance(text, str):
        return text
    lines = textwrap.wrap(text, width=width, break_long_words=False, replace_whitespace=False)
    if len(lines) > max_lines:
        lines = lines[:max_lines-1] + [lines[max_lines-1] + "…"]
    return "\n".join(lines)

# Add nodes (labels for participants/topics; subtopics hover-only to reduce clutter)
for n, attrs in G.nodes(data=True):
    ntype = attrs.get("type", "concept")
    style = type_style.get(ntype, type_style["concept"])
    color = tint_by_sentiment(style["color"], attrs.get("sentiment"))
    show_label = ntype in ("participant", "topic")
    label_text = wrap_label(attrs.get("label", n), width=28, max_lines=3) if show_label else ""

    title_bits = [f"<b>label</b>: {attrs.get('label', n)}"]
    for k in ("type", "introduced_by", "sentiment"):
        if attrs.get(k):
            title_bits.append(f"<b>{k}</b>: {attrs[k]}")

    net.add_node(
        n,
        label=label_text,
        title="<br>".join(title_bits),
        shape=style["shape"],
        color=color,
        level=attrs.get("level", 1)
    )

# Add edges
for u, v, attrs in G.edges(data=True):
    rel = attrs.get("relation") or "related"
    tb = [f"<b>relation</b>: {rel}"]
    if attrs.get("stance"): tb.append(f"<b>stance</b>: {attrs['stance']}")
    if attrs.get("when"):   tb.append(f"<b>at</b>: {attrs['when']}")
    if attrs.get("by"):     tb.append(f"<b>by</b>: {attrs['by']}")
    net.add_edge(u, v, color=edge_color.get(rel, "#757575"), title="<br>".join(tb), arrows="to")

# Physics already disabled; users can toggle in the configure panel if needed
# net.repulsion(...) is optional here; leave off to respect hierarchical spacing

# 4) Save & render (inline in Jupyter and as HTML file)
out_file = "mindmap_interactive_clean.html"
net.show(out_file)
print(f"Rendered above; also saved to {out_file}")


mindmap_interactive_clean.html
Rendered above; also saved to mindmap_interactive_clean.html


In [8]:
# write_mindmap_html.py
# Creates a standalone interactive HTML ("mindmap_app.html") from mindmap.json

import json, html
from pathlib import Path

DATA_PATH = "mindmap.json"
OUT_HTML  = "rough1_mindmap_app.html"

data = json.loads(Path(DATA_PATH).read_text(encoding="utf-8"))

# Topic list for the branch picker
topics = ["All Topics"] + [t["topic"] for t in data.get("main_topics", []) if t.get("topic")]

# Embed the data as JSON inside the HTML
data_js = json.dumps(data, ensure_ascii=False)

html_doc = f"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Conversation Mind Map</title>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<!-- Vis Network (render engine) -->
<link rel="preconnect" href="https://unpkg.com">
<script src="https://unpkg.com/vis-network@9.1.7/dist/vis-network.min.js"></script>
<link href="https://unpkg.com/vis-network@9.1.7/styles/vis-network.min.css" rel="stylesheet"/>

<style>
  :root {{
    --bg: #ffffff;
    --fg: #263238;
    --edge: #CFD8DC;
    --root: #B2DFDB;
    --topic: #C5CAE9;
    --subtopic: #BBDEFB;
    --leaf: #E1F5FE;
    --participant: #D1C4E9;
    --highlight: #FFE082;
  }}
  body {{ margin:0; background:var(--bg); color:var(--fg); font-family: Inter, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial, "Apple Color Emoji", "Segoe UI Emoji"; }}
  .toolbar {{
    display:flex; gap:.75rem; flex-wrap:wrap; align-items:center;
    padding:.75rem 1rem; border-bottom:1px solid #ECEFF1; position:sticky; top:0; background:rgba(255,255,255,.97); z-index:10;
  }}
  .toolbar label {{ font-size:.9rem; opacity:.85; }}
  .toolbar input[type="range"] {{ width:160px; vertical-align:middle; }}
  .toolbar input[type="text"] {{ width:220px; padding:.4rem .55rem; border:1px solid #ECEFF1; border-radius:8px; }}
  .toolbar select {{ padding:.35rem .5rem; border:1px solid #ECEFF1; border-radius:8px; }}
  .pill {{ display:inline-block; padding:.15rem .45rem; font-size:.75rem; border-radius:999px; background:#EEF2F7; margin-left:.35rem; }}
  #graph {{ height: calc(100vh - 64px); }}
  .note {{ padding:.35rem 1rem; font-size:.85rem; color:#607D8B; border-top:1px dashed #ECEFF1; }}
</style>
</head>

<body>
  <div class="toolbar">
    <label>Branch
      <select id="topic"></select>
    </label>

    <label>Depth
      <input id="depth" type="range" min="1" max="8" step="1" value="3"/>
      <span id="depthVal" class="pill">3</span>
    </label>

    <label>Wrap
      <input id="wrap" type="range" min="14" max="36" step="1" value="22"/>
      <span id="wrapVal" class="pill">22</span>
    </label>

    <label>Layout
      <select id="direction">
        <option value="LR" selected>Left → Right</option>
        <option value="UD">Top → Down</option>
      </select>
    </label>

    <label><input id="showParts" type="checkbox"/> Show participants</label>
    <label><input id="hideX" type="checkbox" checked/> Hide cross-links</label>

    <label>Highlight
      <input id="search" type="text" placeholder="type to highlight…"/>
    </label>

    <span id="stats" class="pill"></span>
  </div>

  <div id="graph"></div>
  <div class="note">Tip: Pick a single branch (topic) and keep depth at 2–3 for lowest cognitive load. Use the highlight box to quickly find context.</div>

<script>
  // ----- Embedded Data -----
  const DATA = {data_js};

  // ----- UI -----
  const topicSel   = document.getElementById('topic');
  const depthEl    = document.getElementById('depth');
  const depthVal   = document.getElementById('depthVal');
  const wrapEl     = document.getElementById('wrap');
  const wrapVal    = document.getElementById('wrapVal');
  const dirEl      = document.getElementById('direction');
  const showParts  = document.getElementById('showParts');
  const hideX      = document.getElementById('hideX');
  const searchEl   = document.getElementById('search');
  const statsEl    = document.getElementById('stats');

  // Populate topics
  const TOPICS = ["All Topics", ...(DATA.main_topics||[]).map(t => t.topic).filter(Boolean)];
  TOPICS.forEach(t => {{
    const opt = document.createElement('option');
    opt.value = t; opt.textContent = t;
    topicSel.appendChild(opt);
  }});

  depthEl.addEventListener('input', () => depthVal.textContent = depthEl.value);
  wrapEl .addEventListener('input', () => wrapVal.textContent  = wrapEl.value);

  // ----- Utility -----
  function wrapLabel(s, width) {{
    s = (s||"").trim().replace(/\\s+/g, " ");
    if (s.length <= width) return s;
    const out = []; let line = [], ln = 0;
    for (const w of s.split(" ")) {{
      const extra = line.length ? 1 : 0;
      if (ln + w.length + extra > width) {{
        out.push(line.join(" ")); line=[w]; ln = w.length;
      }} else {{ line.push(w); ln += w.length + extra; }}
    }}
    if (line.length) out.push(line.join(" "));
    return out.join("<br>");
  }}

  function tint(hex, sentiment) {{
    if (!sentiment || sentiment === "neutral") return hex;
    const r = parseInt(hex.slice(1,3),16), g=parseInt(hex.slice(3,5),16), b=parseInt(hex.slice(5,7),16);
    let R=r,G=g,B=b;
    if (sentiment === "positive") {{ R=Math.min(255, Math.round(r*1.10)); G=Math.min(255, Math.round(g*1.10)); B=Math.min(255, Math.round(b*1.10)); }}
    if (sentiment === "negative") {{ R=Math.round(r*0.80); G=Math.round(g*0.80); B=Math.round(b*0.80); }}
    return "#" + [R,G,B].map(v => v.toString(16).padStart(2,"0")).join("");
  }}

  // ----- Graph Build (NotebookLM-style) -----
  function buildData(opts) {{
    const {{
      selectedTopic = "All Topics",
      maxDepth = 3,
      hideCrosslinks = true,
      showParticipants = false,
      wrap = 22
    }} = opts;

    const nodes = new vis.DataSet();
    const edges = new vis.DataSet();

    const addNode = (id, role, label, attrs={{}}) => {{
      if (!id) return;
      if (nodes.get(id)) return;

      const palette = {{
        root:       {{ color: getComputedStyle(document.documentElement).getPropertyValue('--root').trim(),       shape: 'box' }},
        topic:      {{ color: getComputedStyle(document.documentElement).getPropertyValue('--topic').trim(),      shape: 'box' }},
        subtopic:   {{ color: getComputedStyle(document.documentElement).getPropertyValue('--subtopic').trim(),   shape: 'box' }},
        leaf:       {{ color: getComputedStyle(document.documentElement).getPropertyValue('--leaf').trim(),       shape: 'box' }},
        participant:{{ color: getComputedStyle(document.documentElement).getPropertyValue('--participant').trim(),shape: 'diamond' }},
      }};
      const base = palette[role] || palette.leaf;
      const color = tint(base.color, attrs.sentiment);

      nodes.add({{
        id,
        label: wrapLabel(label||id, wrap),
        title: label||id,
        color,
        shape: base.shape,
        margin: 10,
        font: {{ multi: "html", size: 14 }}
      }});
    }};

    const addEdge = (from, to) => {{ if (from && to) edges.add({{ from, to }}); }};

    const root = DATA.root || DATA.title || "Mind Map";
    addNode(root, "root", root);

    if (showParticipants && Array.isArray(DATA.participants)) {{
      for (const p of DATA.participants) {{
        if (!p || !p.name) continue;
        addNode(p.name, "participant", p.name);
        addEdge(root, p.name);
      }}
    }}

    function addTopicBranch(topicObj) {{
      const tname = topicObj.topic;
      addNode(tname, "topic", tname, {{ sentiment: topicObj.sentiment }});
      addEdge(root, tname);

      const subs = topicObj.subtopics || [];
      for (const s of subs) {{
        const sname = s.subtopic;
        addNode(sname, "subtopic", sname, {{ sentiment: s.sentiment }});
        addEdge(tname, sname);

        // Optional leaves: entities/notes under subtopic
        const leaves = (s.entities || []).map(x => (typeof x === 'string' ? x : (x && (x.name || x.text)))) 
                    .concat((s.notes || []).map(x => (typeof x === 'string' ? x : (x && (x.name || x.text))))).filter(Boolean);
        for (const leaf of leaves) {{
          addNode(leaf, "leaf", leaf);
          addEdge(sname, leaf);
        }}

        if (!hideCrosslinks && Array.isArray(s.discussed_by)) {{
          for (const who of s.discussed_by) {{
            addNode(who, "participant", who);
            addEdge(sname, who);
          }}
        }}
      }}
    }}

    if (selectedTopic === "All Topics") {{
      for (const t of (DATA.main_topics || [])) addTopicBranch(t);
    }} else {{
      const t = (DATA.main_topics || []).find(x => x.topic === selectedTopic);
      if (t) addTopicBranch(t);
    }}

    // Depth pruning from root
    const adj = new Map(); nodes.forEach(n => adj.set(n.id, []));
    edges.forEach(e => {{ if (adj.has(e.from)) adj.get(e.from).push(e.to); }});
    const keep = new Set([root]); let frontier = [root], d=0;
    while (frontier.length && d < maxDepth) {{
      const nxt = [];
      for (const u of frontier) {{
        const kids = adj.get(u) || [];
        for (const v of kids) if (!keep.has(v)) {{ keep.add(v); nxt.push(v); }}
      }}
      frontier = nxt; d += 1;
    }}
    // Filter nodes/edges to kept
    const n2 = new vis.DataSet(nodes.get().filter(n => keep.has(n.id)));
    const kset = new Set(n2.getIds());
    const e2 = new vis.DataSet(edges.get().filter(e => kset.has(e.from) && kset.has(e.to)));

    return {{ nodes: n2, edges: e2 }};
  }}

  // ----- Graph Render -----
  const container = document.getElementById('graph');
  let network = null;

  function render() {{
    const opts = {{
      selectedTopic: topicSel.value || "All Topics",
      maxDepth: parseInt(depthEl.value, 10),
      hideCrosslinks: hideX.checked,
      showParticipants: showParts.checked,
      wrap: parseInt(wrapEl.value, 10),
    }};
    const data = buildData(opts);

    const options = {{
      layout: {{
        hierarchical: {{
          enabled: true,
          direction: dirEl.value,           // "LR" or "UD"
          sortMethod: "hubsize",
          levelSeparation: 230,
          nodeSpacing: 210,
          treeSpacing: 280
        }}
      }},
      physics: {{ enabled: false }},
      nodes: {{
        shape: "box",
        color: {{
          border: "#ECEFF1",
          highlight: {{ border: "#90CAF9", background: "#E3F2FD" }}
        }},
        widthConstraint: {{ maximum: 260 }}
      }},
      edges: {{
        smooth: {{ type: "continuous" }},
        color: {{ color: getComputedStyle(document.documentElement).getPropertyValue('--edge').trim() }},
        arrows: {{ to: {{ enabled: false }} }}
      }},
      interaction: {{ hover: true, tooltipDelay: 80 }}
    }};

    // reset & render
    container.innerHTML = "";
    network = new vis.Network(container, data, options);

    // simple highlight (client-side) based on search box
    const q = (searchEl.value || "").trim().toLowerCase();
    if (q) {{
      const ids = data.nodes.getIds();
      for (const id of ids) {{
        const n = data.nodes.get(id);
        const lbl = (n.title || "").toLowerCase();
        if (lbl.includes(q)) {{
          data.nodes.update({{ id, color: getComputedStyle(document.documentElement).getPropertyValue('--highlight').trim() }});
        }}
      }}
    }}

    // update stats
    statsEl.textContent = data.nodes.length + " nodes · " + data.edges.length + " edges";
  }}

  // ----- Events -----
  [topicSel, depthEl, wrapEl, dirEl, showParts, hideX].forEach(el => el.addEventListener('input', render));
  searchEl.addEventListener('input', render);

  // Init controls defaults
  topicSel.value = "All Topics";
  render();
</script>
</body>
</html>
"""

Path(OUT_HTML).write_text(html_doc, encoding="utf-8")
print(f"✅ Wrote {OUT_HTML}. Open it in your browser.")


✅ Wrote rough1_mindmap_app.html. Open it in your browser.


#### Below is the Finalise Code we using

In [9]:
# write_mindmap_html_with_edge_attribution.py
# Standalone HTML with NotebookLM-style layout and SPEAKER ATTRIBUTION ON EDGES (no participant nodes)

import json
from pathlib import Path

DATA_PATH = "mindmap.json"
OUT_HTML  = "rough2_mindmap_app.html"

data = json.loads(Path(DATA_PATH).read_text(encoding="utf-8"))
data_js = json.dumps(data, ensure_ascii=False)

html_doc = f"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Conversation Mind Map</title>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<script src="https://unpkg.com/vis-network@9.1.7/dist/vis-network.min.js"></script>
<link href="https://unpkg.com/vis-network@9.1.7/styles/vis-network.min.css" rel="stylesheet"/>

<style>
  :root {{
    --bg: #ffffff;
    --fg: #263238;
    --edge: #CFD8DC;
    --root: #B2DFDB;
    --topic: #C5CAE9;
    --subtopic: #BBDEFB;
    --leaf: #E1F5FE;
    --highlight: #FFE082;
  }}
  body {{ margin:0; background:var(--bg); color:var(--fg); font-family: Inter, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial; }}
  .toolbar {{
    display:flex; gap:.75rem; flex-wrap:wrap; align-items:center;
    padding:.75rem 1rem; border-bottom:1px solid #ECEFF1; position:sticky; top:0; background:rgba(255,255,255,.97); z-index:10;
  }}
  .toolbar label {{ font-size:.9rem; opacity:.9; }}
  .toolbar input[type="range"] {{ width:160px; vertical-align:middle; }}
  .toolbar input[type="text"] {{ width:220px; padding:.4rem .55rem; border:1px solid #ECEFF1; border-radius:8px; }}
  .toolbar select {{ padding:.35rem .5rem; border:1px solid #ECEFF1; border-radius:8px; }}
  .pill {{ display:inline-block; padding:.15rem .45rem; font-size:.75rem; border-radius:999px; background:#EEF2F7; margin-left:.35rem; }}
  #graph {{ height: calc(100vh - 64px); }}
  .note {{ padding:.35rem 1rem; font-size:.85rem; color:#607D8B; border-top:1px dashed #ECEFF1; }}
</style>
</head>

<body>
  <div class="toolbar">
    <label>Branch
      <select id="topic"></select>
    </label>

    <label>Depth
      <input id="depth" type="range" min="1" max="8" step="1" value="3"/>
      <span id="depthVal" class="pill">3</span>
    </label>

    <label>Wrap
      <input id="wrap" type="range" min="14" max="36" step="1" value="22"/>
      <span id="wrapVal" class="pill">22</span>
    </label>

    <label>Layout
      <select id="direction">
        <option value="LR" selected>Left → Right</option>
        <option value="UD">Top → Down</option>
      </select>
    </label>

    <label><input id="hideX" type="checkbox" checked/> Hide cross-links</label>
    <label><input id="edgeLabels" type="checkbox" checked/> Show speaker labels on edges</label>

    <label>Highlight
      <input id="search" type="text" placeholder="type to highlight…"/>
    </label>

    <span id="stats" class="pill"></span>
  </div>

  <div id="graph"></div>
  <div class="note">Tip: Choose one branch (topic) and keep depth at 2–3 for clarity. Toggle “Show speaker labels” to see who introduced/discussed each node.</div>

<script>
  // -------- Embedded Data --------
  const DATA = {data_js};

  // -------- UI refs --------
  const topicSel   = document.getElementById('topic');
  const depthEl    = document.getElementById('depth');
  const depthVal   = document.getElementById('depthVal');
  const wrapEl     = document.getElementById('wrap');
  const wrapVal    = document.getElementById('wrapVal');
  const dirEl      = document.getElementById('direction');
  const hideX      = document.getElementById('hideX');
  const edgeLabels = document.getElementById('edgeLabels');
  const searchEl   = document.getElementById('search');
  const statsEl    = document.getElementById('stats');

  // Populate topics
  const TOPICS = ["All Topics", ...(DATA.main_topics||[]).map(t => t.topic).filter(Boolean)];
  TOPICS.forEach(t => {{
    const opt = document.createElement('option');
    opt.value = t; opt.textContent = t;
    topicSel.appendChild(opt);
  }});

  depthEl.addEventListener('input', () => depthVal.textContent = depthEl.value);
  wrapEl .addEventListener('input', () => wrapVal.textContent  = wrapEl.value);

  // -------- Utils --------
  function wrapLabel(s, width) {{
    s = (s||"").trim().replace(/\\s+/g, " ");
    if (s.length <= width) return s;
    const out = []; let line = [], ln = 0;
    for (const w of s.split(" ")) {{
      const extra = line.length ? 1 : 0;
      if (ln + w.length + extra > width) {{
        out.push(line.join(" ")); line=[w]; ln = w.length;
      }} else {{ line.push(w); ln += w.length + extra; }}
    }}
    if (line.length) out.push(line.join(" "));
    return out.join("<br>");
  }}

  function tint(hex, sentiment) {{
    if (!sentiment || sentiment === "neutral") return hex;
    const r = parseInt(hex.slice(1,3),16), g=parseInt(hex.slice(3,5),16), b=parseInt(hex.slice(5,7),16);
    let R=r,G=g,B=b;
    if (sentiment === "positive") {{ R=Math.min(255, Math.round(r*1.10)); G=Math.min(255, Math.round(g*1.10)); B=Math.min(255, Math.round(b*1.10)); }}
    if (sentiment === "negative") {{ R=Math.round(r*0.80); G=Math.round(g*0.80); B=Math.round(b*0.80); }}
    return "#" + [R,G,B].map(v => v.toString(16).padStart(2,"0")).join("");
  }}

  function truncateList(list, maxItems=3) {{
    if (!Array.isArray(list)) return "";
    const items = list.filter(Boolean);
    if (items.length <= maxItems) return items.join(", ");
    return items.slice(0, maxItems).join(", ") + " +" + (items.length - maxItems);
  }}

  // -------- Build graph data (edge attribution) --------
  function buildData(opts) {{
    const {{
      selectedTopic = "All Topics",
      maxDepth = 3,
      hideCrosslinks = true,
      wrap = 22,
      showEdgeLabels = true
    }} = opts;

    const nodes = new vis.DataSet();
    const edges = new vis.DataSet();

    const palette = {{
      root:     getComputedStyle(document.documentElement).getPropertyValue('--root').trim(),
      topic:    getComputedStyle(document.documentElement).getPropertyValue('--topic').trim(),
      subtopic: getComputedStyle(document.documentElement).getPropertyValue('--subtopic').trim(),
      leaf:     getComputedStyle(document.documentElement).getPropertyValue('--leaf').trim()
    }};

    const addNode = (id, role, label, sentiment) => {{
      if (!id || nodes.get(id)) return;
      const color = tint(palette[role] || palette.leaf, sentiment);
      nodes.add({{
        id,
        label: wrapLabel(label||id, wrap),
        title: label||id,
        color,
        shape: "box",
        margin: 10,
        font: {{ multi: "html", size: 14 }}
      }});
    }};

    const addEdge = (from, to, label, title) => {{
      if (!from || !to) return;
      const e = {{ from, to }};
      if (showEdgeLabels && label) e.label = label;
      if (title) e.title = title;
      edges.add(e);
    }};

    const root = DATA.root || DATA.title || "Mind Map";
    addNode(root, "root", root);

    // Edge attribution for topics: "introduced by ..."
    function addTopicBranch(topicObj) {{
      const tname = topicObj.topic;
      addNode(tname, "topic", tname, topicObj.sentiment);

      let topicEdgeLabel = "";
      let topicEdgeTitle = "";
      const introBy = topicObj.introduced_by;
      const introAt = topicObj.introduced_at;
      if (introBy) {{
        topicEdgeLabel = "introduced by " + introBy;
        topicEdgeTitle = "<b>introduced by</b>: " + introBy + (introAt ? "<br><b>at</b>: " + introAt : "");
      }}
      addEdge(root, tname, topicEdgeLabel, topicEdgeTitle);

      // Subtopics: attribution from discussed_by / stance goes on the edge Topic→Subtopic
      for (const s of (topicObj.subtopics || [])) {{
        const sname = s.subtopic;
        addNode(sname, "subtopic", sname, s.sentiment);

        const whoList = (s.discussed_by || []).filter(Boolean);
        const stance  = s.stance;
        const labelWho = truncateList(whoList, 2);
        let subEdgeLabel = "";
        let subEdgeTitle = "";
        if (labelWho || stance) {{
          const parts = [];
          if (labelWho) parts.push("discussed by " + labelWho);
          if (stance)   parts.push("stance: " + stance);
          subEdgeLabel = parts.join(" · ");
          subEdgeTitle = parts.map(p => "<b>" + p.split(":")[0] + "</b>: " + (p.split(":")[1] || "").trim()).join("<br>");
        }}
        addEdge(tname, sname, subEdgeLabel, subEdgeTitle);

        // Optional leaves (entities/notes) under Subtopic (no attribution here to keep it clean)
        const entities = Array.isArray(s.entities) ? s.entities : [];
        const notes    = Array.isArray(s.notes) ? s.notes : [];
        const leaves   = [];
        for (const x of entities) {{
          if (typeof x === 'string') leaves.push(x);
          else if (x && (x.name || x.text)) leaves.push(x.name || x.text);
        }}
        for (const y of notes) {{
          if (typeof y === 'string') leaves.push(y);
          else if (y && (y.name || y.text)) leaves.push(y.name || y.text);
        }}
        for (const leaf of leaves) {{
          addNode(leaf, "leaf", leaf, null);
          addEdge(sname, leaf, "", "");
        }}
      }}
    }}

    if (selectedTopic === "All Topics") {{
      for (const t of (DATA.main_topics || [])) addTopicBranch(t);
    }} else {{
      const t = (DATA.main_topics || []).find(x => x.topic === selectedTopic);
      if (t) addTopicBranch(t);
    }}

    // Cross-link relationships (attribution on those edges too), unless hidden
    if (!hideCrosslinks) {{
      for (const r of (DATA.relationships || [])) {{
        const frm = r.from, to = r.to; if (!frm || !to) continue;
        // Ensure nodes exist (type inference is minimal here)
        addNode(frm, "leaf", frm, null);
        addNode(to,  "leaf", to,  null);
        let lbl = "";
        let ttl = "";
        if (r.type) lbl = r.type;
        const by = r.initiated_by, at = r.initiated_at;
        const extras = [];
        if (by) extras.push("by " + by);
        if (at) extras.push("at " + at);
        if (extras.length) lbl = (lbl ? lbl + " · " : "") + extras.join(" ");
        if (r.type) ttl += "<b>type</b>: " + r.type;
        if (by)     ttl += (ttl ? "<br>" : "") + "<b>by</b>: " + by;
        if (at)     ttl += (ttl ? "<br>" : "") + "<b>at</b>: " + at;
        addEdge(frm, to, lbl, ttl);
      }}
    }}

    // Depth prune (root-out)
    const adj = new Map(); nodes.forEach(n => adj.set(n.id, []));
    edges.forEach(e => {{ if (adj.has(e.from)) adj.get(e.from).push(e.to); }});
    const keep = new Set([root]); let frontier = [root], d=0;
    while (frontier.length && d < maxDepth) {{
      const nxt = [];
      for (const u of frontier) {{
        const kids = adj.get(u) || [];
        for (const v of kids) if (!keep.has(v)) {{ keep.add(v); nxt.push(v); }}
      }}
      frontier = nxt; d += 1;
    }}
    const n2 = new vis.DataSet(nodes.get().filter(n => keep.has(n.id)));
    const kept = new Set(n2.getIds());
    const e2 = new vis.DataSet(edges.get().filter(e => kept.has(e.from) && kept.has(e.to)));

    return {{ nodes: n2, edges: e2 }};
  }}

  // -------- Render --------
  const container = document.getElementById('graph');
  let network = null;

  function render() {{
    const opts = {{
      selectedTopic: topicSel.value || "All Topics",
      maxDepth: parseInt(depthEl.value, 10),
      hideCrosslinks: hideX.checked,
      wrap: parseInt(wrapEl.value, 10),
      showEdgeLabels: edgeLabels.checked
    }};
    const data = buildData(opts);

    const options = {{
      layout: {{
        hierarchical: {{
          enabled: true,
          direction: dirEl.value,   // "LR" or "UD"
          sortMethod: "hubsize",
          levelSeparation: 230,
          nodeSpacing: 210,
          treeSpacing: 280
        }}
      }},
      physics: {{ enabled: false }},
      nodes: {{
        shape: "box",
        color: {{
          border: "#ECEFF1",
          highlight: {{ border: "#90CAF9", background: "#E3F2FD" }}
        }},
        widthConstraint: {{ maximum: 260 }}
      }},
      edges: {{
        smooth: {{ type: "continuous" }},
        color: {{ color: getComputedStyle(document.documentElement).getPropertyValue('--edge').trim() }},
        arrows: {{ to: {{ enabled: false }} }},
        font: {{ align: "top", size: 11, color: "#546E7A", background: "#FAFAFA" }}
      }},
      interaction: {{ hover: true, tooltipDelay: 80 }}
    }};

    container.innerHTML = "";
    network = new vis.Network(container, data, options);

    // Client-side highlight
    const q = (searchEl.value || "").trim().toLowerCase();
    if (q) {{
      const ids = data.nodes.getIds();
      for (const id of ids) {{
        const n = data.nodes.get(id);
        const lbl = (n.title || "").toLowerCase();
        if (lbl.includes(q)) {{
          data.nodes.update({{ id, color: getComputedStyle(document.documentElement).getPropertyValue('--highlight').trim() }});
        }}
      }}
    }}

    statsEl.textContent = data.nodes.length + " nodes · " + data.edges.length + " edges";
  }}

  // Events
  [topicSel, depthEl, wrapEl, dirEl, hideX, edgeLabels].forEach(el => el.addEventListener('input', render));
  searchEl.addEventListener('input', render);

  // Init
  topicSel.value = "All Topics";
  render();
</script>
</body>
</html>
"""

Path(OUT_HTML).write_text(html_doc, encoding="utf-8")
print(f"✅ Wrote {OUT_HTML}. Open it in your browser.")


✅ Wrote rough2_mindmap_app.html. Open it in your browser.
